In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def get_balanceByDays(row,days):
    if row['DiasMora'] > days:
        return row['SaldoColocaciones']
    else:
        return 0

In [ ]:
#Cargado Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/02. Riesgo Instituciones/01. Riesgo de crédito/data_blog.xlsx')
df['Mayor8Dias'] = df.apply(get_balanceByDays,axis=1,args=[8])
df['Mayor30Dias'] = df.apply(get_balanceByDays,axis=1,args=[30])

In [ ]:
df.head()

In [ ]:
def get_date_int(df, column) :
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [ ]:
anio_cierre, mes_cierre, dia_cierre = get_date_int(df, 'FechaCierre')
anio_cosecha, mes_cosecha, dia_cosecha = get_date_int(df,'FechaDesembolso')
years_diff = anio_cierre - anio_cosecha
months_diff = mes_cierre - mes_cosecha
df['AlturaVida'] = years_diff * 12 + months_diff

In [ ]:
def ind_mora(row):
    return row['Mayor30Dias']/row['SaldoColocaciones']

In [ ]:
situacion = pd.pivot_table(df[(df['FechaCierre']=='2022-07-31') & (df['FechaDesembolso']>='2021-07-01')],
                index=pd.Grouper(key='FechaDesembolso',freq='1ME'),
                values=["SaldoColocaciones","Mayor30Dias"],
                aggfunc='sum'
            )
situacion.reset_index()
situacion["indice"] = situacion.apply(ind_mora,axis=1)

In [ ]:
cohort_data = df[df['FechaDesembolso']>='2021-07-01'].groupby(
                [pd.Grouper(key='FechaDesembolso',freq='1ME'),'AlturaVida']
              )['Mayor30Dias'].apply(np.sum)
cohort_data = cohort_data.reset_index()

cohort_mora = cohort_data.pivot_table(
                index = 'FechaDesembolso',
                columns = 'AlturaVida',
                values = 'Mayor30Dias'
            )

In [ ]:
cohort_data_des = df[df['FechaDesembolso']>='2021-07-01'].groupby(
                    [pd.Grouper(key='FechaDesembolso',freq='1ME'),'AlturaVida']
                  )['MontoDesembolso'].apply(np.sum)

cohort_data_des = cohort_data_des.reset_index()

cohort_counts_des = cohort_data_des.pivot_table(
                     index = pd.Grouper(key='FechaDesembolso',freq='1ME'),
                     columns = 'AlturaVida',
                     values = 'MontoDesembolso'
                    )

In [ ]:
indVintage = cohort_mora.divide(cohort_counts_des.iloc[:,0], axis = 0)

indVintage = (indVintage * 100).round(4)

indVintage.index = indVintage.index.strftime('%Y-%m')

In [ ]:
plt.figure(figsize = (15,8))
ax = sns.heatmap(indVintage, annot = True, cmap="OrRd", fmt='.2f',linewidths=.5,cbar=False)
ax.xaxis.tick_top()
ax.xaxis.label_position = 'top'
ax.set(title='Cosechas de crédito SK',ylabel='Fecha de cosecha')

In [ ]:
indVintage.T[['2021-07','2021-08','2021-09','2021-10','2021-11']].plot(figsize=(12,6))
plt.title('Ind. Cosecha')
plt.xticks(np.arange(1, 12.1, 1))
plt.xlim(1, 12)
plt.ylabel('% de Ind Cosecha')